In [1]:
from utils import get_openai_client
OPEN_API_KEY, client = get_openai_client()

# API 연결 테스트
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 함수 작성


리뷰 JSON 파일 로드

In [4]:
import json

with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[:3]

[{'review': '깨끗하고 너무 좋았어요', 'stars': 5, 'date': '22시간 전'},
 {'review': '위치도 아주좋고 숙소도 깨끗하고 좋아요~', 'stars': 5, 'date': '2일 전'},
 {'review': '잘쉬고놀다갑니다  감사', 'stars': 5, 'date': '6일 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [5]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(177, 43)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [6]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_END]',
 '[REVIEW_START]근처 볼거리 놀거리가 가까이 있어 편했으나 \n조계사 고층뷰는 의미 없이 배정해 주었으며\n객실 내 청결상태가 별로였음 욕조 머리카락, 수건의 먼지등\n재방문 의사 없음[REVIEW_END]',
 '[REVIEW_START]2월21일(수) 오전7시경, 카운터에 계시던 남자 직원 불친절\n방음 최악[REVIEW_END]']

In [7]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:100]

'[REVIEW_START]인사동, 북촌동, 종각 등등 근처에 갈곳두 많구\r\n숙소두 깨끗하구 너무 좋아요\r\n엘베를 2번 타야하는게 살짝 번거롭지만\r\n그거빼구 다 좋아요[REVIEW_'

전처리 함수 작성

In [9]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
good[:100]

'[REVIEW_START]깨끗하고 너무 좋았어요[REVIEW_END]\n[REVIEW_START]위치도 아주좋고 숙소도 깨끗하고 좋아요~[REVIEW_END]\n[REVIEW_STAR'

### 평가용 함수 작성

평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
  - 비교하는 방식 vs. 점수 매기는 방식
  - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
  - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summaries, accommodations)

In [12]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

### Baseline 모델 개발

In [8]:
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

In [14]:
# PROMPT_BASELINE 정의
PROMPT_BASELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""

# client 다시 정의
from utils import get_openai_client
OPEN_API_KEY, client = get_openai_client()

reviews, _ = preprocess_reviews(path='./res/reviews.json')

def summarize(reviews, prompt, temperature=0.0, model='gpt-3.5-turbo-0125', client=client):
    prompt = prompt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

숙소는 깨끗하고 위치도 좋았으며 편안한 휴식을 즐겼다고 감사의 말을 전했다.


In [15]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'

In [11]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BASELINE).choices[0].message.content, summary_real_20240526).choices[0].message.content)

Assistant A provides a concise summary that accurately reflects the user reviews. It mentions the cleanliness, good location, and the fact that the guests enjoyed their stay and expressed gratitude. This summary is directly relevant to the reviews provided.

Assistant B, on the other hand, includes a lot of additional information that is not present in the user reviews. While the response is detailed and creative, it introduces elements such as specific tourist attractions, staff service, and nearby amenities, which are not mentioned in the original reviews. This makes the summary less accurate and less relevant to the given reviews.

Therefore, Assistant A better follows the user’s instructions by providing a summary that is helpful, relevant, accurate, and directly based on the user reviews.

[[A]]


### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 Prompt에 대해 temperature 높여서 평가 진행

In [17]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['위치가 좋고 깨끗한 숙소에서 편안하게 쉬다 감사했다.',
 '숙소는 깨끗하고 위치도 좋았으며 쉬고 놀기에 좋았다는 감사한 리뷰입니다.',
 '깨끗하고 위치 좋은 숙소로 편안하게 쉬고 놀기에 좋았다고 감사의 마음을 전했다.',
 '이 숙소는 깨끗하고 좋은 위치에 있어서 휴식을 즐기기 좋은 곳이었다.',
 '숙소는 깨끗하고 위치도 좋았으며 잘 쉬고 즐거운 시간을 보냈다고 감사한 리뷰입니다.',
 '모든 숙소 리뷰는 매우 만족스러웠고, 깨끗하고 좋은 위치에 위치해 있었으며 휴식을 즐길 수 있었다.',
 '깨끗하고 위치도 좋고 잘 쉬고 놀았다고 감사했다.',
 '숙소는 깨끗하고 위치가 좋았으며 잘 쉬고 놀 수 있었던 만족스러운 숙박이었습니다.',
 '숙소는 깨끗하고 아주 좋았으며 위치도 좋았다. 잘 쉬고 놀다 온다고 감사의 마음을 전했다.',
 '모든 숙박객이 깨끗하고 만족스러운 숙소에 만족했다.']

In [19]:
from tqdm import tqdm


def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print('Evaluation Error')

    return a_cnt, b_cnt, draw_cnt

wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20240526 for _ in range(len(summaries_baseline))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:37<00:00,  3.77s/it]

Wins: 10, Losses: 0, Ties: 0


### 모델 고도화 1 - 조건들 명시

In [20]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:32<00:00,  3.27s/it]

Wins: 10, Losses: 0, Ties: 0


### 모델 고도화 2 - 입력 데이터의 품질 증가

In [24]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date

        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]

    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews()

In [25]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:51<00:00,  5.19s/it]

Wins: 3, Losses: 7, Ties: 0


### 모델 고도화 3 - Few-Shot Prompting

In [34]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:55<00:00,  5.51s/it]

Wins: 1, Losses: 9, Ties: 0


In [35]:
summaries

['이 숙소는 전반적으로 깔끔하고 아늑한 분위기를 가졌으며 친절한 직원들이 있어서 좋았습니다. 또한 위치가 편리하고 주변에 편의시설이 많아서 재밌게 머물렀습니다. 다만 욕실 청결상태가 조금 아쉬웠지만 전체적으로 만족스러운 숙박이었습니다. 또한 다음에도 방문을 고려하며 싶은 숙소였습니다. 감사합니다.',
 '1. 이번 여행에서 머물렀던 숙소는 전반적으로 깔끔하고 아늑한 분위기를 느낄 수 있었습니다.\n2. 직원들의 친절한 응대와 청결한 시설로 편안한 휴식을 취할 수 있어서 좋았습니다.\n3. 바다를 바라볼 수 있는 객실에서의 풍경은 정말 멋있었고, 재방문하고 싶은 숙소였습니다.',
 '숙소는 전반적으로 깔끔하고 편안한 분위기를 느낄 수 있었으며, 친절한 스태프들이 만족스러운 서비스를 제공했습니다. 특히, 맛있는 조식과 편리한 위치가 좋았습니다. 다음에 또 방문할 의향이 있습니다.',
 '1. 객실이 깔끔하고 직원들의 친절함이 눈에 띄었습니다.\n2. 위치가 시내 중심가에 가까워 편리했고 주변에 맛집들도 많았습니다.\n3. 아침 식사가 풍성하고 맛있어서 여행 일정을 시작하는 데 좋았습니다.\n4. 다음에 또 방문하고 싶은 호텔이었습니다.',
 '1. 숙소는 깨끗하고 아늑한 분위기가 좋았습니다. 직원들도 친절했고 위치도 편리했습니다.\n2. 침대가 푹신해서 편안하게 잠을 청했고, 조식 메뉴도 다양하고 맛있었습니다.\n3. 전체적으로 만족스러운 숙박이었고, 재방문 의사가 있습니다. 감사합니다.',
 '이 숙소는 깔끔하고 시설이 잘 갖춰져 있어서 편안한 숙박을 할 수 있었습니다. 직원들의 친절함과 서비스도 만족스러웠고, 위치도 편리해서 다시 방문하고 싶은 마음이 들었습니다. 전체적으로 좋은 체험이었고, 추천할 만한 숙소입니다.',
 '숙소는 전반적으로 청결하고 아늑한 분위기를 가지고 있었습니다. 직원들도 친절하여 편안한 숙박을 할 수 있었습니다. 또한 위치가 편리하여 주변 관광지에 접근하기에 좋은 장점이 있었습니다. 다음에 또 방문하고 싶은 숙소였습니다. 강력 추천합니

In [47]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_1shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [01:08<00:00,  6.83s/it]

Wins: 9, Losses: 1, Ties: 0


In [37]:
summaries

['1. "가족과 함께한 일정에서 어느 것 하나 빠짐없이 잘 즐기고 왔습니다!! 밤 늦은 시간에는 좀 한적하긴 하지만 차라리 조용해서 숙박하는데 더 좋았어요"\n2. "평일 6시쯤 수영했는데 사람 거의 없어서 정말 좋았어요! 방도 깨끗하고 가성비 좋아요. 베개는 좀 높아서 불편했네요. 그래도 소음이나 냄새 문제는 하나도 없었어요"\n3. "방음이 잘 안되는 것만 빼면 청결도 친절도 위치 부대시설 모두 만족스럽습니다!"\n4. "생일날 여자친구 퇴근이 늦어져 회사 인근에서 머물렀는데 서비스도 좋고 시설도 상당히 만족스러웠습니다. 다음에 또 오게 된다면 재방문 의향 있습니다!"\n5. "깔끔하고 맘에 듭니다! 주차장도 넓어요~ 다음에 또 오고싶네요"\n6. "수영도하고 숙박도하니 가성비 매우좋네요 상가에서 먹고 거북이까페가고 하느라 돈 더많이 쓰긴 했지만 이불 포근하고 깨끗하고 좋았어요~~"\n7. "포근하고 깨끗하고 쾌적했습니다! 지내다가 좋아서 계속 추가 연박했네요 ㅋㅋ 재방문의사 100%!!"\n8. "객실 깨끗하고 수영장도 좁지만 좋아용 사우나도 있고 잘 놀다 왔어용~~"\n9. "정말 좋은 호텔입니다. 이용때마다 느끼지만 직원분들도 친절하시고 방도 깨끗합니다."\n10. "주변이 9시만 지나도 다 닫지만 객실은 너무 좋아서 휴식하기 좋아요"\n\n요약:\n이 숙소는 청결하고 친절한 서비스를 제공하며, 수영장 시설이 만족스러우며 가성비가 좋다는 평가를 받고 있습니다. 주변에는 편리한 시설들이 잘 갖춰져 있어 손님들의 만족도가 높은 편입니다. 다만 일부 이용객은 방음이나 베개의 높이 등 몇 가지 불편한 점을 언급했습니다. 전반적으로 다양한 이용객들이 재방문 의사를 표명하며, 가족 단위의 방문객들에게 특히 추천되는 숙소로 손꼽히고 있습니다.',
 '1. "가족과 함께한 일정에서 어느것 하나 빠짐없이 잘 즐기고 왔습니다!! 밤 늦은 시간에는 좀 한적하긴 하지만 차라리 조용해서 숙박하는데 더 좋았어요"\n- 가족이 함께하는 여행을 즐기기에 좋은 조용한 숙박 시설\

In [38]:
reviews_1shot, _ = preprocess_reviews('./res/ninetree_pangyo.json')
summary_1shot = summarize(reviews_1shot, prompt, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

In [43]:
reviews_2shot, _ = preprocess_reviews('./res/ninetree_yongsan.json')
summary_2shot = summarize(reviews_2shot, prompt_1shot, temperature=0.0, model='gpt-4-turbo-2024-04-09').choices[0].message.content

prompt_2shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다. 다음은 리뷰들과 요약 예시입니다.

예시 리뷰들 1:
{reviews_1shot}
예시 요약 결과 1:
{summary_1shot}

예시 리뷰들 2:
{reviews_2shot}
예시 요약 결과 2:
{summary_2shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt_2shot, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20240526 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:34<00:00,  3.45s/it]

Wins: 10, Losses: 0, Ties: 0


In [46]:
summaries

['청결하고 쾌적한 환경을 제공하는 이 숙소는 직원들의 친절함과 서비스 품질이 높게 평가되고 있습니다. 객실의 깨끗함과 넓은 공간, 조식의 맛과 다양성, 그리고 수영장 시설이 인기를 끌고 있습니다. 또한 용산역과 인접해 교통이 편리하고 주변에 쇼핑 및 식당이 많아 편리하다는 점도 좋은 평가를 받고 있습니다. 일부 이용객은 주차 비용이 별도로 청구되거나 엘리베이터 대기 시간이 길다는 점을 지적했으나, 전반적으로 만족도가 높아 재방문 의사를 표현하는 리뷰들이 많이 있습니다.',
 '1. 숙소는 가족과 함께한 여행에 적합하며, 수영장 시설이 좋다는 평가와 청결한 객실이 인상적이었다.\n2. 조식의 맛과 객실 내부의 청결함이 만족스러웠고, 직원의 친절한 응대가 언급되었다.\n3. 호캉스로 이용하기에 좋은 숙소로 평가되었으며, 수영장에 대한 긍정적인 평가가 많이 나왔다.\n4. 위치가 용산역과 가깝고, 주변에 맛집과 놀거리가 많아서 편리하다는 의견이 많았다.\n5. 객실의 깨끗함과 침구의 편안함이 언급되었으며, 다시 방문하고 싶다는 의사를 표현하는 리뷰가 많았다.',
 '1. 가족과 함께 머무르기 좋은 숙소로 호평받고 있으며, 수영장 시설이 만족스럽다는 평가가 많이 나옵니다.\n2. 객실과 침구의 청결도가 높게 평가되고 있으며, 프런트 직원의 친절함도 언급되고 있습니다.\n3. 주변 환경이 조용하고 위치가 편리하다는 의견이 많으며, 숙소 내의 편의시설과 서비스도 만족스러운 것으로 보입니다.\n4. 조식의 맛과 다양성, 수영장 시설, 편의시설 등이 고객들에게 긍정적인 인상을 준 것으로 파악됩니다.',
 '1. "가족과 함께한 일정에서 어느 것 하나 빠짐없이 잘 즐기고 왔습니다!! 밤 늦은 시간에는 좀 한적하긴 하지만 차라리 조용해서 숙박하는데 더 좋았어요"\n2. "평일 6시쯤 수영했는데 사람 거의 없어서 정말 좋았어요! 방도 깨끗하고 가성비 좋아요. 베개는 좀 높아서 불편했네요. 그래도 소음이나 냄새 문제는 하나도 없었어요"\n3. "방음이 잘 안되는 것만 빼면 청결도